# Display some images at different steps of the analysis 

The aim of this notebook is to enable the teams to vizualise images and actions that are performed through the analysis:
- Image as received in the original file
- Pump detection and end of the pump detection
- Working zone delimitation

In [ ]:
#!pip install opencv-python

In [ ]:
# Imports 
import sys
import os
import cv2
import json
import pandas as pd
import random
import sys
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from math import sqrt
import glob
sys.path.insert(0, "..")
from utils_analytics import *

## 1. Import Data and display some images containing pumps

In [ ]:
# Selecting some images that do contain pump hoses 
data_df = pd.read_pickle('../inputs/table_labels_new.pkl')
df_pump = data_df[(data_df['classTitle'] == 'Concrete_pump_hose') & (data_df['filename'].str.startswith('2020')) ]

In [ ]:
# Creating directories with images with pump_hoses
#! mkdir /home/jovyan/eleven2020/inputs/images_pump/
#! mkdir /home/jovyan/eleven2020/inputs/labels_pump/

In [ ]:
i = 0 
for filename in df_pump[df_pump['classTitle'] == 'Concrete_pump_hose']['filename']:
    if i == 150 :
        break
    else :
        i+=1
        #shutil.copy('/home/jovyan/eleven2020/inputs/images/'+filename[:-5], '/home/jovyan/eleven2020/inputs/images_pump/'+filename[:-5])
        #shutil.copy('/home/jovyan/eleven2020/inputs/labels/'+filename, '/home/jovyan/eleven2020/inputs/labels_pump/'+filename)

In [ ]:
# Showing images with pump hoses and their detections 
show_image_labeled_sample(20,'/home/jovyan/eleven2020/inputs/images_pump/', '/home/jovyan/eleven2020/inputs/labels_pump/')

## 2. Display some images with pump extremity detected 

To find the extremity of the pump, we designed many rules that applied to some perspectives of the pump. 
Although we used only the first rule in our solution, the idea (with more time) would be to ponderate the three rules into a single one in order to adapt as much as possible to the position of the pump in each image. 

### a. First rule to find extremity : find thinnest part of polygon (what has been applied so far)

In [ ]:
# Print a few images with extremity
for file_name in list(df_pump.filename)[2:3]:
    print(file_name)

    polygon = list(df_pump[df_pump['filename']==file_name]['ext_points'])[0]

    extremity = find_thinnest_part(polygon)

    show_image_with_extremity('../inputs/images_pump/'+file_name[:-5], extremity, '../inputs/labels_pump/'+file_name)


### b. Second rule : Find lowest point of polygon

In [ ]:
# Print a few images with extremity
for file_name in list(df_pump.filename)[100:101]:
    print(file_name)

    polygon = list(df_pump[df_pump['filename']==file_name]['ext_points'])[0]
    print(polygon)
    extremity = find_lowest_point(polygon)
    print(extremity)

    show_image_with_extremity('../inputs/images_pump/'+file_name[:-5], extremity, '../inputs/labels_pump/'+file_name)


### c. Third rule : find point at the bottom of the most vertical line

In [ ]:
# Defining some functions aiming at detecting the most vertical line of the polygon 
def slope_btw_two_points(pt1, pt2):
    if ((pt2[0] == 0) & (pt1[0]== 0)) :
        return -1
    elif pt2[0]-pt1[0] != 0 :
        slope = (pt2[1]-pt1[1])/(pt2[0]-pt1[0])
        return slope
    else :
        return 10000
        

def find_lowest_vertical_point(polygon):
    """Returns the coordinates of the lowest point on the most vertical line of the polygon

    Parameters
    ----------
    polygon : list
        list of coordinates of the polygon
    """
    most_vertical_slope = 0.001
    for point_A in polygon:
        for point_B in polygon: 
            if point_A != point_B : 
                slope = abs(slope_btw_two_points(point_A, point_B))
                if slope > most_vertical_slope:
                    most_vertical_slope = slope
                    lowest_point = find_lowest_point([point_A, point_B])
    print(point_A, point_B)
    return lowest_point

In [ ]:
# Printing a few images with extremity
for file_name in list(df_pump.filename)[112:113]:
    print(file_name)

    polygon = list(df_pump[df_pump['filename']==file_name]['ext_points'])[0]
    
    extremity = find_lowest_vertical_point(polygon)


    show_image_with_extremity('../inputs/images_pump/'+file_name[:-5], extremity, '../inputs/labels_pump/'+file_name)

### Additional step : Trying to smooth polygons for better results

Please note that this section is an intent that has not been applied afterwards, but that could help in getting even better results.

In [ ]:
def distance(pt1, pt2):
    distance = sqrt( ((pt1[0]-pt2[0])**2)+((pt1[1]-pt2[1])**2) )
    return distance

def remove_small_segments(polygon, treshold):
    new_polygon=[polygon[0]]
    for index in range(len(polygon)):
        if index == len(polygon) -1 :
            break
        else: 
            if distance(new_polygon[-1], polygon[index+1])>treshold:
                new_polygon.append(polygon[index+1])
    return new_polygon

In [ ]:
slope_treshold = 0.05
look_further = 5

def liss(poly):
    new_poly = [poly[0]]
    new_index = 0
    for index in range(len(poly)):
        if index >= len(poly) - look_further+1:
            break
        elif index < new_index:
            continue
        else:
            go_to = []
            for i in range(index+1, index+look_further):
                s_1 = slope_btw_two_points(poly[index], poly[i])
                s_2 = slope_btw_two_points(poly[index], poly[i+1])
                #print(s_2-s_1)
                if abs(s_2-s_1) < slope_treshold:
                    #maybe add condition if not ridiculous
                    go_to = poly[i+1]
                    new_index = index+i+1
            if go_to == []: 
                go_to = poly[index+1]
                new_index = index + 1
            
            #print(go_to)
        new_poly.append(go_to)
    return new_poly

In [ ]:
# Selecting one image to show the result 
img_name = '2020_07_30_14_24_13.jpg.json'
DIR_IMAGES= '../inputs/images_pump/'
DIR_LABELS = '../inputs/labels_pump/'

In [ ]:
# Applying the smoothness algorithm to one image 
random.seed(189)
import random
#for i in range(5):
#img_name = random.choice(list(df_pump.filename))
poly = list(df_pump[(df_pump['filename']==img_name) & (df_pump['classTitle']=='Concrete_pump_hose')]['ext_points'])[0]
print(len(poly))
if len(poly)>10 :
    new_poly = remove_small_segments(poly, 15)
    new_poly = liss(new_poly)
elif(len(remove_small_segments(poly, 15)))>10:
    new_poly = liss(new_poly)
else :
    new_poly = poly

img = mpimg.imread(DIR_IMAGES+img_name[:-5])
img_copy = img.copy()
#out = cv2.fillPoly(img_copy, pts=[np.array([poly])], color = (239,132,91))
out = cv2.fillPoly(img_copy, pts=[np.array([new_poly])], color = (75, 37, 109))
plt.figure(figsize=(10, 10))
plt.imshow(out)

## 3. Print the zone and extended zone

In [ ]:
# Selecting a datatable in which the zone has already been computed
data_df = pd.read_pickle('../inputs/table_test.pkl')

In [ ]:
# Choosing an image to display 
img_name = '2020_07_30_09_22_44.jpg.json'
data_df[(data_df['filename']==img_name) & (data_df['classTitle']=='Concrete_pump_hose')]

In [ ]:
# Showing the image of the zone / extended zone and the workers that are inside and outside of the zone 
print('This zone has been created with fake values just to show the extended zone and the workers')
draw_pump_worker_anlysis(data_df,'../inputs/analytics/images/2020_07_30_09_22_44.jpg', '../inputs/analytics/labels/2020_07_30_09_22_44.jpg.json')